# Mamba Architecture
<img src="assets/mamba_architecture.png"/>

as we can see from the architecture the new is 
**SSM (Structured State Space Model)** : 
are at the core of Mamba, so it is important to note how they work. You can think of them as the replacement for the self attention mechanism in a transformer,<br/><br/>

A state space model takes in a 1D input sequence, maps to a N-D latent space and then projects back to a 1D output sequence.<br/><br/>
<img src="assets/statespace.png" width="50%"/>
<br/><br/>
**SSMs** like S4 can be defined as through these equations: 

\begin{align*}
h'(t) &= Ah(t) + Bx(t) \\
y(t) &= Ch(t)
\end{align*}

the first equation : 
<img src="assets/ht.png"/>

the second equation : 
<img src="assets/yt.png"/>
